In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

In [ ]:
# Set paths
dataset_dir = "processed_images"  # Replace with your dataset path

In [ ]:
# Load dataset from directories
dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode="int",
    shuffle=True
)

In [ ]:
# # Preprocess images
# def preprocess_image(image, label):
#     # Resize and normalize
#     image = tf.image.resize(image, (128, 128))
#     image = image / 255.0

#     # Convert TensorFlow tensor to NumPy array for OpenCV
#     image_np = image.numpy()  # Convert tensor to NumPy array
#     image_np = (image_np * 255).astype(np.uint8)  # Scale to 0-255 for OpenCV
#     image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)  # Convert to BGR format

#     # Convert to HSV for skin detection
#     hsv_image = cv2.cvtColor(image_np, cv2.COLOR_BGR2HSV)
#     lower_skin = (0, 20, 70)  # Lower range for skin detection
#     upper_skin = (20, 255, 255)  # Upper range for skin detection
#     skin_mask = cv2.inRange(hsv_image, lower_skin, upper_skin)

#     # Apply morphological operations
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
#     skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

#     # Apply mask to the image
#     masked_image = cv2.bitwise_and(image_np, image_np, mask=skin_mask)

#     # Detect edges using Canny
#     edges = cv2.Canny(masked_image, 100, 200)

#     # Convert processed image back to tensor
#     edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)  # Convert to RGB format
#     edges_tensor = tf.image.convert_image_dtype(edges / 255.0)  # Normalize and convert to tensor

#     # Return processed tensor and label
#     return edges_tensor, label

In [ ]:
def rgb_to_grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

In [ ]:
# # Prepare dataset
# # Map preprocessing function to dataset
# def preprocess_wrapper(image, label):
#     image, label = tf.py_function(preprocess_image, [image, label], [tf.float32, tf.int64])
#     print(label)
#     image.set_shape((128, 128, 3))  # Ensure image has the correct shape
#     label.set_shape([])  # Ensure label has the correct shape
#     return image, label

dataset = dataset.map(rgb_to_grayscale, num_parallel_calls=tf.data.AUTOTUNE)

validation_split = 0.2
train_size = int((1 - validation_split) * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

for images, labels in train_dataset.take(1):
    print(f"Images shape: {images.shape}")
    print(f"Labels type: {type(labels)}")

In [ ]:
# Build the model
def build_model(input_shape=(128, 128, 1), num_classes=35):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
# Train the model
model = build_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_dataset, validation_data=val_dataset, epochs=1)

In [ ]:
# Save the trained model to a file
model.save("sign_language_model.h5")

In [ ]:
from tensorflow.keras.models import load_model # type: ignore

model = load_model("sign_language_model.h5")

In [ ]:
import cv2
import numpy as np
import os

class_names = sorted(os.listdir(dataset_dir))
class_names = class_names[1:]

cap = cv2.VideoCapture(0)  # Open webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_frame = cv2.resize(gray_frame, (128, 128)) 
    edges = cv2.Canny((resized_frame * 255).astype(np.uint8), threshold1=100, threshold2=200)  # Apply Canny edge detection
    normalized_edges = edges / 255.0 # Resize to model input size
    input_frame = tf.convert_to_tensor(normalized_edges, dtype=tf.float32)
    input_frame = np.expand_dims(input_frame, axis=-1)  # Add channel dimension for grayscale
    input_frame = np.expand_dims(input_frame, axis=0)       # Add batch dimension

    # Predict the class
    predictions = model.predict(input_frame)
    predicted_class = class_names[np.argmax(predictions)]

    # Display prediction on the frame
    cv2.putText(frame, f"Prediction: {predicted_class}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Webcam", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("test_2.jpg")
ori = image.copy()

In [ ]:
# gray_frame = cv2.cvtColor(input_test, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
# _, binary_image = cv2.threshold(gray_frame, 100, 255, cv2.THRESH_BINARY)

# # Find contours in the binary image
# contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Create a blank mask
# mask = np.zeros_like(gray_frame)

# # Draw the largest contour (assuming it represents the hand) on the mask
# # if contours:
# #     largest_contour = max(contours, key=cv2.contourArea)
# #     cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

#  # Refine contour selection
# selected_contour = None
# min_area = 100  # Minimum contour area threshold
# max_area = 50000  # Maximum contour area threshold
# image_center = np.array([gray_frame.shape[1] // 2, gray_frame.shape[0] // 2])

# for contour in contours:
#     # Filter by area
#     area = cv2.contourArea(contour)
#     if not (min_area < area < max_area):
#         continue

#     # Calculate bounding box aspect ratio
#     x, y, w, h = cv2.boundingRect(contour)
#     aspect_ratio = float(w) / h
#     if not (0.5 < aspect_ratio < 2.0):  # Acceptable aspect ratio range
#         continue

#     # Calculate contour centroid distance from image center
#     M = cv2.moments(contour)
#     if M["m00"] != 0:
#         cx, cy = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])
#         distance_from_center = np.linalg.norm(image_center - np.array([cx, cy]))
#         if selected_contour is None or distance_from_center < best_distance:
#             selected_contour = contour
#             best_distance = distance_from_center

image = cv2.resize(image, (128, 128))
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the lower and upper range for skin color in HSV
lower_skin = (0, 20, 70)   # Adjust as per the skin tone
upper_skin = (20, 255, 255)

# Create a mask for skin color
skin_mask = cv2.inRange(hsv_image, lower_skin, upper_skin)

# Apply morphological operations to clean up the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

# Blur the mask for smoother edges
skin_mask = cv2.GaussianBlur(skin_mask, (5, 5), 0)

# Detect edges in the mask
edges = cv2.Canny(skin_mask, 100, 200)

# Find contours in the edge-detected mask
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the largest contour on the original image
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    cv2.drawContours(image, [largest_contour], -1, (0, 255, 0), 2)

# Plot the results
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(ori, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Skin Mask")
plt.imshow(skin_mask, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Detected edges")
plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.show()

# # Apply the mask to the original grayscale image
# masked_image = cv2.bitwise_and(gray_frame, gray_frame, mask=mask)
# cv2.imshow('image', masked_image) # Resize to model input size
# cv2.waitKey(0)


In [ ]:

# edges = cv2.Canny((masked_image * 255).astype(np.uint8), threshold1=500, threshold2=600, L2gradient=True)  # Apply Canny edge detection
# resized_frame = cv2.resize(edges, (128, 128))
normalized_edges = edges / 255.0
input_frame = tf.convert_to_tensor(normalized_edges, dtype=tf.float32)
input_frame = np.expand_dims(input_frame, axis=-1)  # Add channel dimension for grayscale
input_frame = np.expand_dims(input_frame, axis=0) 
pred = model.predict(input_frame)

In [ ]:
ind = 0
maxi = 0
for i in range(len(pred[0])):
    if pred[0, i] > maxi:
        maxi = pred[0, i]
        ind = i
print(ind+1)

In [ ]:
pred.shape

In [ ]:
pred

In [ ]:
type(input_frame)

In [ ]:
contours